In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np
from datetime import datetime

# Import data
path = "appointment.csv"

noshow_df = pd.read_csv(path)
noshow_df

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,2.572134e+12,5651768,F,2016-05-03T09:15:35Z,2016-06-07T00:00:00Z,56,MARIA ORTIZ,0,0,0,0,0,1,No
110523,3.596266e+12,5650093,F,2016-05-03T07:27:33Z,2016-06-07T00:00:00Z,51,MARIA ORTIZ,0,0,0,0,0,1,No
110524,1.557663e+13,5630692,F,2016-04-27T16:03:52Z,2016-06-07T00:00:00Z,21,MARIA ORTIZ,0,0,0,0,0,1,No
110525,9.213493e+13,5630323,F,2016-04-27T15:09:23Z,2016-06-07T00:00:00Z,38,MARIA ORTIZ,0,0,0,0,0,1,No


In [2]:
# Preprocessing
noshow_df['No-show'] = noshow_df['No-show'].map({'Yes':1, 'No':0})

noshow_df = noshow_df.drop(columns=['AppointmentID'])

noshow_df['ScheduledDay'] = pd.to_datetime(noshow_df['ScheduledDay'])
noshow_df['AppointmentDay'] = pd.to_datetime(noshow_df['AppointmentDay'])
noshow_df['ScheduledDay'] = noshow_df['ScheduledDay'].dt.day
noshow_df['AppointmentDay'] = noshow_df['AppointmentDay'].dt.day

noshow_df['PatientId'] = noshow_df['PatientId'].astype(str)



In [3]:
noshow_df.dtypes

PatientId         object
Gender            object
ScheduledDay       int32
AppointmentDay     int32
Age                int64
Neighbourhood     object
Scholarship        int64
Hipertension       int64
Diabetes           int64
Alcoholism         int64
Handcap            int64
SMS_received       int64
No-show            int64
dtype: object

In [4]:
noshow_df.nunique()

PatientId         62299
Gender                2
ScheduledDay         31
AppointmentDay       24
Age                 104
Neighbourhood        81
Scholarship           2
Hipertension          2
Diabetes              2
Alcoholism            2
Handcap               5
SMS_received          2
No-show               2
dtype: int64

In [5]:
noshow_df.isnull().sum()

PatientId         0
Gender            0
ScheduledDay      0
AppointmentDay    0
Age               0
Neighbourhood     0
Scholarship       0
Hipertension      0
Diabetes          0
Alcoholism        0
Handcap           0
SMS_received      0
No-show           0
dtype: int64

In [6]:
# Convert categorical data to numeric with `pd.get_dummies`
noshow_df_dummies = pd.get_dummies(noshow_df)
noshow_df_dummies.dtypes

PatientId                      float64
ScheduledDay                     int32
AppointmentDay                   int32
Age                              int64
Scholarship                      int64
                                ...   
Neighbourhood_SÃO JOSÉ            bool
Neighbourhood_SÃO PEDRO           bool
Neighbourhood_TABUAZEIRO          bool
Neighbourhood_UNIVERSITÁRIO       bool
Neighbourhood_VILA RUBIM          bool
Length: 94, dtype: object

In [7]:
noshow_df_dummies.isnull().sum()

PatientId                      0
ScheduledDay                   0
AppointmentDay                 0
Age                            0
Scholarship                    0
                              ..
Neighbourhood_SÃO JOSÉ         0
Neighbourhood_SÃO PEDRO        0
Neighbourhood_TABUAZEIRO       0
Neighbourhood_UNIVERSITÁRIO    0
Neighbourhood_VILA RUBIM       0
Length: 94, dtype: int64

In [8]:
# Split our preprocessed data into our features and target arrays
X = noshow_df_dummies.drop('No-show', axis=1)
y = noshow_df_dummies['No-show']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
len(X_train_scaled[0])

93

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 20
hidden_nodes_layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                1880      
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 2,101
Trainable params: 2,101
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
2764/2764 [==============================] - 3s 968us/step - loss: 0.5061 - accuracy: 0.7887
Epoch 2/100
2764/2764 [==============================] - 3s 1ms/step - loss: 0.4911 - accuracy: 0.7980
Epoch 3/100
2764/2764 [==============================] - 3s 1ms/step - loss: 0.4853 - accuracy: 0.7979
Epoch 4/100
2764/2764 [==============================] - 3s 1ms/step - loss: 0.4773 - accuracy: 0.7974
Epoch 5/100
2764/2764 [==============================] - 3s 1ms/step - loss: 0.4723 - accuracy: 0.7975
Epoch 6/100
2764/2764 [==============================] - 3s 1ms/step - loss: 0.4692 - accuracy: 0.7977
Epoch 7/100
2764/2764 [==============================] - 3s 1ms/step - loss: 0.4663 - accuracy: 0.7976
Epoch 8/100
2764/2764 [==============================] - 3s 1ms/step - loss: 0.4648 - accuracy: 0.7979
Epoch 9/100
2764/2764 [==============================] - 3s 1ms/step - loss: 0.4626 - accuracy: 0.7979
Epoch 10/100
2764/2764 [==============================] - 3s 1ms/step -

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

691/691 - 1s - loss: 0.4587 - accuracy: 0.7953 - 636ms/epoch - 920us/step
Loss: 0.4586722254753113, Accuracy: 0.7952591776847839
